In [37]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

23/03/19 13:35:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/19 13:35:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Names

Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [38]:
# import time

# start = time.time()
# filepath = "dbfs:/FileStore/tables/Files/names.csv"
filepath = "names.csv"
names_df = spark.read.format("csv") \
          .option("header","true") \
          .option("inferSchema","true") \
          .load(filepath)
# end = time.time()

# display(names_df)

**Add current timestamp**

In [39]:
from pyspark.sql.functions import *

# names_df = names_df.withColumn("execution_time", lit(str(end - start)))
names_df = names_df.withColumn("current_timestamp", current_timestamp())

display(names_df.select("current_timestamp"))

DataFrame[current_timestamp: timestamp]

**Add height column in feet**

In [40]:
names_df = names_df.withColumn("height (feet)", col("height") / 2.54 / 12)

#  display(names_df.select("height (feet)"))

**Most popular name**

In [41]:
names_df.groupBy("name") \
    .agg(count("*")) \
    .sort(desc("count(1)")) \
    .show()

+-------------------+--------+
|               name|count(1)|
+-------------------+--------+
|     David Campbell|       4|
|      David Bradley|       4|
|     James Morrison|       3|
|      Robert Gordon|       3|
|       Frank Miller|       3|
|      Michael Jones|       3|
|   Michael Williams|       3|
|       Charles Lane|       3|
|         David Rose|       3|
|      Paul Williams|       3|
|Aleksandr Mikhaylov|       3|
|         John Moore|       3|
|     David Mitchell|       3|
|       Justin Kelly|       3|
|        Ian Roberts|       3|
|        Tony Martin|       3|
|   Charles Williams|       3|
|       James Murray|       3|
|          John Wood|       3|
|   William Marshall|       3|
+-------------------+--------+
only showing top 20 rows



**Add column with actors age**

Get birth timestamp

In [42]:
# date_of_birth

names_df = names_df.withColumn("birth", regexp_replace("date_of_birth","\.", "-")) \
               .withColumn("birth", regexp_replace("birth", "[A-Za-z,]", "")) \
               .withColumn("birth_timestamp", to_timestamp(col("birth"))) 
               # .select(col("birth"), when(to_timestamp(col("birth"),"yyyy-MM-dd").isNotNull, to_timestamp(col("birth"),"yyyy-MM-dd")))
# display(names_df.select("birth_timestamp"))

Get death timestamp

In [43]:
# date_of_death
names_df = names_df.withColumn("death", regexp_replace("date_of_death","\.", "-")) \
               .withColumn("death", regexp_replace("death", "[A-Za-z,]", "")) \
               .withColumn("death_timestamp", to_timestamp(col("death"), "dd-mm-yyyy"))
               # .select(col("birth"), when(to_timestamp(col("birth"),"yyyy-MM-dd").isNotNull, to_timestamp(col("birth"),"yyyy-MM-dd")))
# display(names_df.select("death_timestamp"))

Calculate age

In [44]:
names_df = names_df.withColumn("Age", months_between(col("death_timestamp"), col("birth_timestamp")) / 12)
# names_df = names_df.withColumn("Age", months_between(current_date(), col("birth_timestamp")) / 12)

# display(names_df.select(col("Age")))

**Drop bio and death_details**

In [45]:
names_df = names_df.drop("bio", "death_details")

# display(names_df)

**Normalize column names**

In [46]:
columns = names_df.columns

Capitalize

In [47]:
columns = list(map(str.capitalize, columns))

Remove underscores

In [48]:
columns = list(map(lambda x: x.replace("_", " "), columns))

Set new columns

In [49]:
names_df = names_df.toDF(*columns)

# display(names_df)

**Sort DataFrame**

In [50]:
names_df = names_df.sort("Name")

# display(names_df)

# Movies

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [51]:
# filepath = "dbfs:/FileStore/tables/Files/movies.csv"
filepath = "movies.csv"
movies_df = spark.read.format("csv") \
          .option("header","true") \
          .option("inferSchema","true") \
          .load(filepath)

display(movies_df)

DataFrame[imdb_title_id: string, title: string, original_title: string, year: string, date_published: string, genre: string, duration: int, country: string, language: string, director: string, writer: string, production_company: string, actors: string, description: string, avg_vote: string, votes: int, budget: string, usa_gross_income: string, worlwide_gross_income: string, metascore: double, reviews_from_users: double, reviews_from_critics: double]

**Current timestamp**

In [52]:
movies_df = movies_df.withColumn("current_timestamp", current_timestamp())

# display(movies_df.select("current_timestamp"))

**Year differnce**

In [53]:
movies_df = movies_df.withColumn("year_diff", months_between(col("current_timestamp"), to_timestamp(col("year"), "yyyy")) / 12)

# display(movies_df.select("year_diff"))

**Numeric budget**

In [54]:
from pyspark.sql.types import IntegerType

movies_df = movies_df.withColumn("budget_num", regexp_replace("budget", "[^0-9.]", "").cast(IntegerType()))

**Drop missing values**

In [55]:
movies_df = movies_df.dropna()

# display(movies_df)

# Ratings

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Dodaj nowe kolumny i policz mean i median dla wartości głosów (1 d 10)
* Dla każdej wartości mean i median policz jaka jest różnica między weighted_average_vote
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long`

In [56]:
# filepath = "dbfs:/FileStore/tables/Files/ratings.csv"
filepath = "ratings.csv"
ratings_df = spark.read.format("csv") \
              .option("header","true") \
              .option("inferSchema","true") \
              .load(filepath)

display(ratings_df)

DataFrame[imdb_title_id: string, weighted_average_vote: double, total_votes: int, mean_vote: double, median_vote: double, votes_10: int, votes_9: int, votes_8: int, votes_7: int, votes_6: int, votes_5: int, votes_4: int, votes_3: int, votes_2: int, votes_1: int, allgenders_0age_avg_vote: double, allgenders_0age_votes: double, allgenders_18age_avg_vote: double, allgenders_18age_votes: double, allgenders_30age_avg_vote: double, allgenders_30age_votes: double, allgenders_45age_avg_vote: double, allgenders_45age_votes: double, males_allages_avg_vote: double, males_allages_votes: double, males_0age_avg_vote: double, males_0age_votes: double, males_18age_avg_vote: double, males_18age_votes: double, males_30age_avg_vote: double, males_30age_votes: double, males_45age_avg_vote: double, males_45age_votes: double, females_allages_avg_vote: double, females_allages_votes: double, females_0age_avg_vote: double, females_0age_votes: double, females_18age_avg_vote: double, females_18age_votes: double,

**Drop missing values**

In [59]:
ratings_df = ratings_df.dropna()

**Current timestamp**

In [21]:
ratings_df = ratings_df.withColumn("current_timestamp", current_timestamp())

# display(ratings_df.select("current_timestamp"))

**Mean votes column**

In [22]:
ratings_df = ratings_df.withColumn("mean_votes", (col("votes_10") + col("votes_9") + col("votes_8") + col("votes_7") + col("votes_6") + col("votes_5") \
                            + col("votes_4") + col("votes_3") + col("votes_2") + col("votes_1")) / 10)

In [23]:
# ratings_df.select("votes_10", "votes_9", "votes_8", "votes_7", "votes_6", "votes_5", "votes_4", "votes_3", "votes_2", "votes_1", "mean_votes").show()

**Median votes column**

In [24]:
import numpy

def median(*args):
    return int(numpy.median([int(x) for x in args]))

# User defined function
udf_median = udf(median, IntegerType())

In [25]:
ratings_df = ratings_df.withColumn("median_votes", udf_median(ratings_df["votes_10"], ratings_df["votes_9"], ratings_df["votes_8"], ratings_df["votes_7"], \
    ratings_df["votes_6"], ratings_df["votes_5"], ratings_df["votes_4"], ratings_df["votes_3"], ratings_df["votes_2"], ratings_df["votes_1"]))

In [26]:
# ratings_df.select("votes_10", "votes_9", "votes_8", "votes_7", "votes_6", "votes_5", "votes_4", "votes_3", "votes_2", "votes_1", "median_votes").show()

**Weighted average score and mean difference**

In [27]:
ratings_df = ratings_df.withColumn("avg_mean_diff", col("weighted_average_vote") - col("mean_votes"))

**Weighted average score and median difference**

In [28]:
ratings_df = ratings_df.withColumn("avg_median_diff", col("weighted_average_vote") - col("median_votes"))

On average women vote higher

In [29]:
ratings_df.select(mean("males_allages_avg_vote"), mean("females_allages_avg_vote")).show()

+---------------------------+-----------------------------+
|avg(males_allages_avg_vote)|avg(females_allages_avg_vote)|
+---------------------------+-----------------------------+
|          5.817172176019604|              6.0394082122788|
+---------------------------+-----------------------------+



**Cast to long**

In [30]:
from pyspark.sql.types import LongType

ratings_df.select(col("total_votes").cast(LongType())).show()

+-----------+
|total_votes|
+-----------+
|        154|
|        589|
|        188|
|        446|
|       2237|
|        484|
|        753|
|        273|
|        198|
|        225|
|        331|
|       1944|
|        948|
|       1349|
|        100|
|        124|
|       1050|
|        187|
|       1768|
|        552|
+-----------+
only showing top 20 rows



In [31]:
spark.stop()